In [1]:
""" Goal: multisig acct with two signers, where either one 
can sign all transactions on behalf of the account.
Start by resetting the sandbox: ./sandbox reset

"""

""" In this cell, we set up the two accounts who will be the two signers
and fund them."""
from random import randint
from settings import Settings
import logging
import dev_setup
import base64
import util
from algosdk.v2client.algod import AlgodClient
from algosdk import encoding
from algosdk.future import transaction
from algosdk.future.transaction import Multisig, MultisigTransaction, wait_for_confirmation
settings = Settings()
logging.basicConfig(level='INFO')

oceanSk = 'sp4SDWmH8Rin0IhPJQq1UMsSR5C0j1IGqzLdcwCMySBVzT8lEUwjwwpS9z6l6dKSg52WWEjRdJDAL+eVt4kvBg=='
oceanAccount = util.AlgoAccount(privateKey=oceanSk)
txn_resp1 = dev_setup.devFundAccount(settings, acctToFund=oceanAccount, microAlgoAmount=1_000_000)

waveSk = 'FCLmrvflibLD6Deu3NNiUQCC9LOWpXLsbMR/cP2oJzH8IT4Zu8vBAhRNsXoWF+2i6q2KyBZrPhmbDCKJD7rBBg=='
waveAccount = util.AlgoAccount(privateKey=waveSk)
txn_resp2 = dev_setup.devFundAccount(settings, acctToFund=waveAccount, microAlgoAmount=1_000_000)
client: AlgodClient = util.getAlgodClient(settings)


INFO:util: ..O5LXUM sending payment of 1.0 algo/s to ..DDX4BI 
 txID: ..FLCYLQ
INFO:util:Got response for ..FLCYLQ
INFO:util: ..KSISJM sending payment of 1.0 algo/s to ..WJ4MII 
 txID: ..J5IQPQ
INFO:util:Got response for ..J5IQPQ


In [3]:
# We have two accounts - ocean and wave. We make a multi-sig account where either can sign

msig = Multisig(version=1, threshold=1, addresses=[oceanAccount.addr, waveAccount.addr])
assert msig.threshold == 1
subsigOcean: transaction.MultisigSubsig = msig.subsigs[0]
assert isinstance(subsigOcean, transaction.MultisigSubsig)
subsigWave: transaction.MultisigSubsig = msig.subsigs[1]
assert subsigOcean.public_key == transaction.MultisigSubsig(encoding.decode_address(oceanAccount.addr)).public_key
assert subsigWave.public_key == transaction.MultisigSubsig(encoding.decode_address(waveAccount.addr)).public_key

assert msig.address()[-6:] == 'KD2SVI'
# Fund the multisig account from a genesis account
gen = dev_setup.devGetGenesisAccounts(settings)[0]
txn = transaction.PaymentTxn(
    sender=gen.addr,
    receiver=msig.address(),
    amt=500_000,
    sp=client.suggested_params()
)
signedTxn = txn.sign(gen.sk)
txid = client.send_transaction(signedTxn)
confirmed_txn = wait_for_confirmation(client, txid, 6)  
assert client.account_info(msig.address())['amount'] >= 500_000


In [4]:
mtx1 = dev_setup.multiMakesNft(msig=msig, subAcct=oceanAccount)

Sent. Waiting for confirmation
4
success sending
{'subsig': [{'pk': 'Vc0/JRFMI8MKUvc+penSkoOdllhI0XSQwC/nlbeJLwY=', 's': 'aadmzFA82fq4c1EoRbYeMBGTBw0eVdaa9dJVTBDIZSV5TDscBiBODaU5cnkNjk6Osy8Jzk6S5oj7kvZMHno0Bg=='}, {'pk': '/CE+GbvLwQIUTbF6FhftouqtisgWaz4ZmwwiiQ+6wQY='}], 'thr': 1, 'v': 1}


In [5]:
mtx2 =  dev_setup.multiMakesNft(msig=msig, subAcct=waveAccount)

AlgoHTTPError: At least one signature didn't pass verification
{'subsig': [{'pk': 'Vc0/JRFMI8MKUvc+penSkoOdllhI0XSQwC/nlbeJLwY=', 's': 'aadmzFA82fq4c1EoRbYeMBGTBw0eVdaa9dJVTBDIZSV5TDscBiBODaU5cnkNjk6Osy8Jzk6S5oj7kvZMHno0Bg=='}, {'pk': '/CE+GbvLwQIUTbF6FhftouqtisgWaz4ZmwwiiQ+6wQY=', 's': 'Dmep0eYO3Fiy8u4e086msMM5s5LClTDTS8f+TUKMe8tVj/8+bhfyE2vo6pW2NZduzFeChKO05y6xA+ZuPuHrAQ=='}], 'thr': 1, 'v': 1}
